In [14]:
!pip install -q ultralytics

In [42]:
#import pytorch and the libraries depending on pytorch
import torch
import torchvision.models as models
#import tensorflow and tf libraries
import tensorflow as tf
#additional libraries for image processing
import numpy as np
#import multithreaded executor
from concurrent.futures import ThreadPoolExecutor
#import cv2 for image processing
import cv2
# import matplot to graph
import matplotlib.pyplot as plt
# OS for general purpose
import os

#import models
from ultralytics import YOLO

In [16]:
#load all models first

#yolo8 for now
def load_yolov8():
    model = YOLO('yolov8n-cls.pt')
    return model

#EfficientNet with keras
def load_efficientnet():
    model = tf.keras.applications.EfficientNetB0(weights='imagenet')
    return model

#Resnet with tf
def load_resnet50():
    model = models.resnet50(pretrained=True)
    model.eval()
    return model

In [17]:
# preprocess the images for each model
def preprocess_image(img, model_name):
    if model_name == 'yolo8':
        # resize image first
        img_resized = cv2.resize(img, (224, 224))
        # distribute 0 -> 1 after normalization
        img_normalized = img_resized / 255.0
        # add the additional dimension so batches can be created
        img_yolo = np.expand_dims(img_resized, axis=0).astype('uint8')
        return img_yolo
    elif model_name == 'efficientnet':
        # resize the image again for effnet
        img_resized = cv2.resize(img, (224, 224))
        # run effnets preprocessing
        img_efficientnet = tf.keras.applications.efficientnet.preprocess_input(img_resized)
        # add the additional dimension so batches can be created
        img_efficientnet = np.expand_dims(img_efficientnet, axis=0)
        return img_efficientnet
    elif model_name == 'resnet50':
        # resize the image again for resnet
        img_resized = cv2.resize(img, (224, 224))
        # run keras preprocessing
        img_resnet = tf.keras.applications.resnet50.preprocess_input(img_resized)
        # add the additional dimension so batches can be created
        img_resnet = np.expand_dims(img_resnet, axis=0)
        return img_resnet
    else:
        # return 0 on invalid input
        return 0

In [40]:
# classify based on model
def classify_image(model, img, model_name):
    if model_name == 'yolo8':
        # Ensure img is a NumPy array with dtype uint8 and correct shape for YOLOv8
        if not isinstance(img, np.ndarray) or img.dtype != np.uint8:
            img = np.array(img, dtype=np.uint8)
        if len(img.shape) == 4:  # Check if batch dimension is already present
            input_img = img
        else:  # Add batch dimension if missing
            input_img = np.expand_dims(img, axis=0)
        results = model(input_img)
        probs = np.array(results[0].probs.data)
        return probs
    elif model_name == 'efficientnet':
        # results should already be formatted
        predictions = model.predict(img)
        return predictions[0]
    elif model_name == 'resnet50':
        with torch.no_grad():
            # copy the images arra
            img_copy = img.copy()
            # convert image to PyTorch tensor
            img_tensor = torch.from_numpy(img_copy).permute(0, 3, 1, 2).float()
            img_tensor = img_tensor.to('cuda' if torch.cuda.is_available() else 'cpu')
            model = model.to(img_tensor.device)
            predictions = model(img_tensor)
        return predictions.cpu().numpy().flatten()

In [44]:
# run the models now
def classify_with_models(dataset):
    # load each model first
    yolov8 = load_yolov8()
    efficientnet = load_efficientnet()
    resnet50 = load_resnet50()

    # results array
    results = []

    # run for each image in the dataset
    for img in dataset:
        # call the preprocessing function for each model
        img_yolo = preprocess_image(img, 'yolo8')
        img_efficientnet = preprocess_image(img, 'efficientnet')
        img_resnet50 = preprocess_image(img, 'resnet50')

        # creat a threadpool to run these simult
        with ThreadPoolExecutor() as executor:
            yolo_pred = executor.submit(classify_image, yolov8, img_yolo, 'yolo8')
            effnet_pred = executor.submit(classify_image, efficientnet, img_efficientnet, 'efficientnet')
            future_resnet_pred = executor.submit(classify_image, resnet50, img_resnet50, 'resnet50')

            # Get results from each model
            yolo_result = yolo_pred.result()
            effnet_result = effnet_pred.result()
            resnet_result = future_resnet_pred.result()

        # average the results
        avg_result = (yolo_result + effnet_result + resnet_result) / 3

        # store the best result for each image along with the confidence
        best_class = np.argmax(avg_result)
        confidence = avg_result[best_class]

        # add to results array
        results.append((best_class, confidence))

    return results

In [20]:
# import the images and add them to an array

# array set
image_dataset = []

for filename in os.listdir('test_images'):
    img_path = os.path.join('test_images', filename)
    img = cv2.imread(img_path)
    if img is not None:
        # resize the image here and remove redundancy in preprocessing
        img_resized = cv2.resize(img, (224, 224))
        image_dataset.append(img_resized)

# verify if the images have been loaded
print("Number of images loaded in: ", len(image_dataset))

Number of images loaded in:  10


In [45]:
# store in results arrray
results = classify_with_models(image_dataset)

for i, (best_class, confidence) in enumerate(results):
    print(f"Image {i + 1} - Best Classified Class: {best_class} with confidence: {confidence}")


0: 224x224 digital_clock 0.03, nematode 0.03, jellyfish 0.02, theater_curtain 0.02, fire_screen 0.02, 59.2ms
Speed: 145.4ms preprocess, 59.2ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

0: 224x224 digital_clock 0.03, nematode 0.03, jellyfish 0.02, theater_curtain 0.02, fire_screen 0.02, 25.0ms
Speed: 151.3ms preprocess, 25.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step

0: 224x224 digital_clock 0.03, nematode 0.03, jellyfish 0.02, theater_curtain 0.02, fire_screen 0.02, 41.6ms
Speed: 163.3ms preprocess, 41.6ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step

0: 224x224 digital_clock 0.03, nematode 0.03, jellyfish 0.02, theater_curtain 0.02, fire_screen 0.02, 45.9ms
Speed: 243.7ms preprocess, 45.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step

0: 224x224